In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain import hub
from langchain.vectorstores import Chroma
import pickle

Document loader

In [80]:
pages = PyPDFLoader("src/Constitution_de_la_republique_tunisiennefr.pdf").load_and_split()
len(pages)

19

Splitter


In [84]:
doc_chunks = []
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    separators=["\nArticle", "\n\n", "\n", ".", "!", "?", ",", " ", ""],
    chunk_overlap=50,
)
chunks = text_splitter.split_documents(pages)
# for i, chunk in enumerate(chunks):
#     doc = Document(
#         page_content=chunk, metadata={"page": doc.metadata["page"], "chunk": i}
#     )
#     doc.metadata["source"] = f"{doc.metadata['page']}-{doc.metadata['chunk']}"
#     doc.metadata["filename"] = "Constitution_de_la_republique_tunisienne" 
#     doc_chunks.append(doc)
len(chunks)

142

Vectorstores

In [4]:
# model =  SentenceTransformer("dangvantuan/sentence-camembert-large")

# sentences = ["Un avion est en train de décoller.",
#           "Un homme joue d'une grande flûte.",
#           "Un homme étale du fromage râpé sur une pizza.",
#           "Une personne jette un chat au plafond.",
#           "Une personne est en train de plier un morceau de papier.",
#           ]

# embeddings = model.encode(sentences)

In [47]:
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/dangvantuan/sentence-camembert-large")
!pip install InstructorEmbedding
from sentence_transformers import SentenceTransformer
from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings

In [85]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

model = SentenceTransformerEmbeddings(model_name="cointegrated/rubert-tiny2")


persist_directory = 'docs/chroma/'
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=model,
    persist_directory=persist_directory
) 

vectordb.persist()
print(vectordb._collection.count())

519


In [ ]:
index.save_local("faiss_index")
# loaded_index = FAISS.load_local("faiss_index", embeddings)

Retrieval


In [86]:
# retriever = index.as_retriever(search_kwargs={"k": 3})
# retriever.invoke("quelle est la réligion de la tunisie?")
query="quel est le couleur du drapeau de la tunisie?"
sim_docs = vectordb.similarity_search_with_score(query, k=3)  
for result in sim_docs:
    print("\n")
    print(result[1])
    print(result[0].page_content)



0.35481518507003784
suite d’une procédure lui offrant les ga ranties indispensables à sa défense.


0.35481518507003784
suite d’une procédure lui offrant les ga ranties indispensables à sa défense.


0.35481518507003784
suite d’une procédure lui offrant les ga ranties indispensables à sa défense.


In [30]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

llm.invoke("do you know tunisian societies laws?")

"\nI'm just an AI, I don't have personal knowledge or experience of specific laws and regulations of any country, including Tunisia. However, I can provide you with some general information about the legal system in Tunisia and direct you to reliable sources for more detailed information.\n\nTunisia is a civil law country, which means that the legal system is based on written codes and laws rather than judicial decisions and customary law. The main source of Tunisian law is the Code of Civil Law (Code de la Justice tunisienne), which was adopted in 1957 and has undergone several amendments since then.\n\nSome key aspects of Tunisian society and laws include:\n\n1. Constitutional monarchy: Tunisia is a constitutional monarchy, with the President serving as the head of state and the Prime Minister as the head of government.\n2. Islamic law: Tunisia has an Islamic majority population, and Islamic law (Shariah) plays a significant role in the country's legal system. However, the country al

In [ ]:
query = "Tell me a joke"

for chunks in llm.stream(query):
    print(chunks)

S
ure
!
 Here
'
s
 one
:




Why
 don
'
t
 scient
ists
 trust
 atoms
?


B
ecause
 they
 make
 up
 everything
!




I
 hope
 that
 brought
 a
 smile
 to
 your
 face
!
 Do
 you
 have
 any
 specific
 topic
 or
 theme
 in
 mind
 for
 a
 jo
ke
,
 and
 I
 can
 try
 to
 come
 up
 with
 something
 tail
ored
 to
 your
 prefer
ences
?

